# Notebook de démonstration et d'évaluation RAG

Ce notebook compare les réponses générées par un modèle baseline et par un pipeline RAG (Retrieval-Augmented Generation) sur un jeu de questions. Les sources utilisées par le RAG sont également affichées pour chaque réponse.

# 0. Importer les bibliothèques nécessaires

In [ ]:
import pandas as pd
import numpy as np
from typing import List, Dict
from pathlib import Path
import os
from dotenv import load_dotenv

# Import du pipeline de traitement
from src.data_processor import DataProcessor

# 1. Lancer la pipeline de préparation des données

On exécute la pipeline complète (scan, clean, chunk, embeddings) pour préparer les données à partir des fichiers bruts.

In [ ]:
# Lancer la pipeline de préparation des données via le script CLI
!python -m src.scripts.build_data_pipeline

## 2. Définir les questions de test

Nous définissons ici une liste de 10 questions représentatives pour l'évaluation du système.

In [ ]:
# Sélection de 10 questions variées pour l'évaluation
questions = [
    "Quand débute l’entre-deux-guerres ?",
    "Quel traité met officiellement fin à la Première Guerre mondiale avec l’Allemagne ?",
    "Dans quelle ville est signé le traité de Versailles ?",
    "Quelle faiblesse structurelle empêche la SDN d’agir efficacement ?",
    "Quelle date marque le début de la Seconde Guerre mondiale ?",
    "Quel objectif idéologique central justifie l’expansion allemande ?",
    "Quel parti politique dirige l’Allemagne à partir de 1933 ?",
    "Quel territoire est remilitarisé par l’Allemagne en 1936 ?",
    "Qui devient chancelier de l’Allemagne en janvier 1933 ?",
    "Quel accord permet à l’Allemagne d’annexer les Sudètes ?"
]

# Affichage des questions
for i, q in enumerate(questions, 1):
    print(f"Q{i}: {q}")


## 3. Définir la fonction de génération de réponse baseline

La fonction ci-dessous simule une génération de réponse baseline (par exemple, un LLM sans accès aux documents).

In [ ]:
# Fonction baseline utilisant les mêmes réglages LLM que run_rag.py ou run_baseline.py
from src.llm_client import LLMClient
import os
from dotenv import load_dotenv

load_dotenv()

api_url = os.getenv("LLM_API_URL", "").strip()
llm_model = os.getenv("LLM_MODEL", "llama3").strip()

client = LLMClient(api_url, llm_model)

BASELINE_SYSTEM_PROMPT = """Tu es un assistant. Réponds de façon concise et factuelle.\nSi tu n'es pas sûr, dis-le explicitement.\n"""

def build_baseline_prompt(question: str) -> str:
    return f"{BASELINE_SYSTEM_PROMPT}\nQuestion: {question}\nRéponse:"

def generate_baseline_answer(question: str) -> str:
    prompt = build_baseline_prompt(question)
    return client.generate(prompt)


## 4. Définir la fonction de génération de réponse RAG avec sources

La fonction suivante simule une génération de réponse RAG et retourne aussi les sources utilisées.

In [ ]:
# Fonction RAG : version hybride dense + BM25 (alignée sur run_rag.py)
import json
import numpy as np
import hnswlib
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import re

# Tokenisation BM25 (identique script)
_WORD_RE = re.compile(r"[A-Za-zÀ-ÖØ-öø-ÿ0-9_]+", re.UNICODE)
def tokenize(text):
    return _WORD_RE.findall(text.lower())

# Chargement des chunks
chunks = []
texts = []
with open('data/chunks.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        item = json.loads(line)
        chunks.append({
            "id": item.get("chunk_id", "?"),
            "text": item["text"],
            "source": item.get("title", item.get("source", "Unknown"))
        })
        texts.append(item["text"])

# Embeddings + index dense
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(texts, convert_to_numpy=True, show_progress_bar=True)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
dim = embeddings.shape[1]
index = hnswlib.Index(space='cosine', dim=dim)
index.init_index(max_elements=len(texts), ef_construction=200, M=16)
index.add_items(embeddings, np.arange(len(texts)))
index.set_ef(50)

# Index BM25
tokenized_corpus = [tokenize(t) for t in texts]
bm25 = BM25Okapi(tokenized_corpus)

# Fusion RRF
def rrf_fuse(dense_ranked_ids, bm25_ranked_ids, rrf_k=60):
    scores = {}
    for rank, idx in enumerate(dense_ranked_ids, start=1):
        scores[idx] = scores.get(idx, 0.0) + 1.0 / (rrf_k + rank)
    for rank, idx in enumerate(bm25_ranked_ids, start=1):
        scores[idx] = scores.get(idx, 0.0) + 1.0 / (rrf_k + rank)
    return [idx for idx, _ in sorted(scores.items(), key=lambda x: x[1], reverse=True)]

def dense_search(query, k):
    q_vec = model.encode([query], convert_to_numpy=True)
    q_vec = q_vec / np.linalg.norm(q_vec, axis=1, keepdims=True)
    labels, _ = index.knn_query(q_vec, k=k)
    return [int(i) for i in labels[0]]

def bm25_search(query, k):
    q_tok = tokenize(query)
    scores = bm25.get_scores(q_tok)
    top_idx = np.argsort(scores)[::-1][:k]
    return [int(i) for i in top_idx]

def hybrid_search(query, k_dense=10, k_bm25=10, k_final=5, rrf_k=60):
    dense_ids = dense_search(query, k=k_dense)
    bm25_ids = bm25_search(query, k=k_bm25)
    fused_ids = rrf_fuse(dense_ids, bm25_ids, rrf_k=rrf_k)[:k_final]
    results = []
    for i in fused_ids:
        results.append({
            "id": chunks[i]["id"],
            "text": chunks[i]["text"],
            "source": chunks[i]["source"]
        })
    return results

BASELINE_SYSTEM_PROMPT = """Tu es un assistant de questions-réponses.\n\nRègles strictes :\n1) Réponds UNIQUEMENT à partir du CONTEXTE fourni.\n2) Si l'information n'est pas présente dans le contexte, réponds : \"Je ne peux pas répondre avec certitude à partir des sources fournies.\"\n3) Ne complète pas avec des connaissances externes.\n4) Donne une réponse concise, puis liste les sources sous forme de puces.\n\nFormat de sortie :\nRéponse : <ta réponse>\n\nSources :\n- <doc_id ou titre> (chunk=<id>)\n- ...\n"""

def build_rag_prompt(question, res):
    context = ""
    for r in res:
        context += f"SOURCE: {r['source']} (chunk={r.get('id','?')})\nTEXT: {r['text']}\n{'-'*60}\n"
    prompt = f"{BASELINE_SYSTEM_PROMPT}\nCONTEXTE :\n{context}\nQUESTION :\n{question}"
    return prompt

def generate_rag_answer(question):
    retrieved_chunks = hybrid_search(question, k_dense=10, k_bm25=10, k_final=3, rrf_k=60)
    prompt = build_rag_prompt(question, retrieved_chunks)
    answer = client.generate(prompt)
    sources = [f"{chunk['source']} (chunk={chunk['id']})" for chunk in retrieved_chunks]
    return answer, sources


## 5. Générer et afficher les réponses baseline

On boucle sur les questions de test et on affiche les réponses générées par la méthode baseline.

In [ ]:
# Génération des réponses baseline
baseline_answers = []
for q in questions:
    ans = generate_baseline_answer(q)
    baseline_answers.append(ans)
    print(f"Q: {q}\nBaseline: {ans}\n")

## 6. Générer et afficher les réponses RAG avec sources

On boucle sur les questions de test et on affiche les réponses générées par la méthode RAG ainsi que les sources associées.

In [ ]:
# Génération des réponses RAG
rag_answers = []
rag_sources = []
for q in questions:
    ans, sources = generate_rag_answer(q)
    rag_answers.append(ans)
    rag_sources.append(sources)
    print(f"Q: {q}\nRAG: {ans}\nSources: {sources}\n")

## 7. Comparer les réponses baseline et RAG

Affichage côte à côte des réponses baseline et RAG pour chaque question afin de faciliter la comparaison.

In [ ]:
# Affichage comparatif des réponses
for i, q in enumerate(questions):
    print(f"Q{i+1}: {q}")
    print(f"  Baseline: {baseline_answers[i]}")
    print(f"  RAG     : {rag_answers[i]}")
    print('-'*60)

## 8. Afficher les sources utilisées pour chaque réponse RAG

Pour chaque question, on affiche la liste des sources utilisées par la méthode RAG.

In [ ]:
# Affichage des sources utilisées pour chaque réponse RAG
for i, q in enumerate(questions):
    print(f"Q{i+1}: {q}")
    print(f"  Sources RAG: {rag_sources[i]}")
    print('-'*60)